# Running the Micom Workflow for the Binary Rhodosporidium - Synechococcus Model

In this notebook we utilize the package `micom` to generate a binary community model for 2 organisms of interest:
- `Rhodosporidium toruloides`
- `Synechococcus elongatus`

This binary consortium allows us to gain insights into the exchanges between the 2 organisms and run FBA experiments.

First off we can import all necessary packages for this notebook.

In [1]:
import pandas as pd
import cobra

from micom import Community
from micom.workflows import build, grow, tradeoff, fix_medium,build_database
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

import os
os.environ["GRB_LICENSE_FILE"]

## Setting Up the Model in MICOM

To begin, we need to import our genome-scale models into `micom`. We have these models saved as `.sbml` files as this form is accepted easily into programs such as `cobrapy` and `micom`.

### Building a Taxonomy

Step #1: Establish a Taxonomy that lists the out the taxonomy for our organisms of interest

In [2]:
Tax= pd.DataFrame(columns=['id','genus','species','reactions','metabolites','sample_id','abundance'])
Tax.loc[len(Tax.index)] = ['Rhodosporidium', 'Rhodosporidium', 'R. toruloides',2398,2051,'One',500]
Tax.loc[len(Tax.index)] = ['Synechococcus', 'Synechococcus','S. elongatus',851,769,'One',500]
Tax

,id,genus,species,reactions,metabolites,sample_id,abundance
0,Rhodosporidium,Rhodosporidium,R. toruloides,2398,2051,One,500
1,Synechococcus,Synechococcus,S. elongatus,851,769,One,500


This taxonomy file lists important information for `micom` down the road, such as the number of `reactions` and `metabolites` in the provided models.

### Building a Database

Step #2: Now we must construct a database for the `.sbml` models to be preprocessed and stored. This is done by supplying `micom` with a file which contains model path locations.

In [3]:
db = pd.read_csv('./man_se_rt.csv')
db

,file,kingdom,phylum,class,order,family,genus,species
0,./Rt_IFO0880.xml,fungi,a,Ustilaginomycetes,b,c,Rhodosporidium,R. toruloides
1,./iJB785.xml,bacteria,Cyanobacteria,Cyanophyceae,Synechococcales,Synechococcaceae,Synechococcus,S. elongatus


In [4]:
build_database(db,'./db_se_rt')

/Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"

[11:01:43] WARNING  The following sinks or demands were forced in the model and have been relaxed since  ]8;id=312460;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/util.py\util.py]8;;\:]8;id=129246;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/util.py#64\64]8;;\
                    they would interfere with MICOM's growth rate estimation: EX_sucr_e                            

,file,kingdom,phylum,class,order,family,genus,species,id,summary_rank
genus,,,,,,,,,,
Rhodosporidium,Rhodosporidium.json,fungi,a,Ustilaginomycetes,b,c,Rhodosporidium,R. toruloides,Rhodosporidium,genus
Synechococcus,Synechococcus.json,bacteria,Cyanobacteria,Cyanophyceae,Synechococcales,Synechococcaceae,Synechococcus,S. elongatus,Synechococcus,genus


In [7]:
db_path = './db_se_rt' 

### Construct Manifest Object

Step #3: Now that we have the Taxonomy and Database constructed we can build our community model. This is done by using the `build()` method in `micom`.

__Note__:

Build Manifest object from Taxonomy DataFrame and the corresponding database directory

Skip this step if manifest has already been built and saved to "models" directory


__IMPORTANT__: Declare the Solver you would like to use for this Community model here:
- osqp (good for smaller models)
- gurobi
- glpk
- cplex
- scipy


In [8]:
manifest = build(Tax, out_folder="models_se_rt", model_db=db_path, cutoff=0.0001, threads=10,solver='gurobi')
manifest

[11:10:12] WARNING  Found existing models for 1 samples. Will skip those. Delete the output folder if   ]8;id=426316;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/workflows/build.py\build.py]8;;\:]8;id=244761;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/workflows/build.py#95\95]8;;\
                    you would like me to rebuild them.                                                             

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmpgo7yv95q.lp
Reading time = 0.02 seconds
: 3107 rows, 7069 columns, 28615 nonzeros


,sample_id,abundance,file,found_taxa,total_taxa,found_fraction,found_abundance_fraction
0,One,500,One.pickle,2.0,2.0,1.0,1.0


## Running the Models with FBA

Now that we have the `manifest`, we can load the model as a `Community` object through `micom`. This will give us some functionality similar to that of `cobrapy`. This can be done with the `load_pickle()` method we imported above through `micom`.

In [10]:
community = load_pickle("models_se_rt/One.pickle")
print(len(community.reactions))

Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmpxvdnn8j7.lp
Reading time = 0.02 seconds
: 3107 rows, 7069 columns, 28615 nonzeros
3534


### Exploring the Model Attributes

Our new variable `community` behaves very similarly to a standard `cobrapy` model. We can explore it's attributes in a similar way as well.

Things such as `reactions` and `metabolites`:

In [11]:
community.reactions

[<Reaction ALCD25yi__Rhodosporidium at 0x32177e4f0>,
 <Reaction MTHFCm__Rhodosporidium at 0x32177e820>,
 <Reaction AMPN__Rhodosporidium at 0x32177e640>,
 <Reaction DAGCPTer_RT__Rhodosporidium at 0x32177efa0>,
 <Reaction PYRt2__Rhodosporidium at 0x321c40d90>,
 <Reaction NNDPRm__Rhodosporidium at 0x321c40af0>,
 <Reaction HMGCOASm__Rhodosporidium at 0x3219f6a60>,
 <Reaction PDE4__Rhodosporidium at 0x321f42130>,
 <Reaction PAPSR__Rhodosporidium at 0x321f52c40>,
 <Reaction FACOAL80p__Rhodosporidium at 0x322254cd0>,
 <Reaction URIH__Rhodosporidium at 0x321e799a0>,
 <Reaction ACOAO4p__Rhodosporidium at 0x322264a00>,
 <Reaction 3IPM3MT__Rhodosporidium at 0x32256cee0>,
 <Reaction UDPGD__Rhodosporidium at 0x32256ca30>,
 <Reaction CYTDK2__Rhodosporidium at 0x3224b4970>,
 <Reaction ACACT4m__Rhodosporidium at 0x3228114c0>,
 <Reaction CHTNDA__Rhodosporidium at 0x3228825b0>,
 <Reaction FACOAL141__Rhodosporidium at 0x322824190>,
 <Reaction ACACT7p__Rhodosporidium at 0x322882f10>,
 <Reaction POLYAO__Rh

In [12]:
community.metabolites

[<Metabolite 2phetoh_c__Rhodosporidium at 0x32177e520>,
 <Metabolite h_c__Rhodosporidium at 0x32177e550>,
 <Metabolite nadp_c__Rhodosporidium at 0x32177e580>,
 <Metabolite nadph_c__Rhodosporidium at 0x32177e5b0>,
 <Metabolite pacald_c__Rhodosporidium at 0x32177e5e0>,
 <Metabolite 10fthf_m__Rhodosporidium at 0x32177e6a0>,
 <Metabolite h2o_m__Rhodosporidium at 0x32177e7f0>,
 <Metabolite h_m__Rhodosporidium at 0x32177e7c0>,
 <Metabolite methf_m__Rhodosporidium at 0x32177e790>,
 <Metabolite ade_c__Rhodosporidium at 0x32177e670>,
 <Metabolite amp_c__Rhodosporidium at 0x32177e610>,
 <Metabolite h2o_c__Rhodosporidium at 0x32177e730>,
 <Metabolite r5p_c__Rhodosporidium at 0x32177e700>,
 <Metabolite 12dgr_RT_r__Rhodosporidium at 0x32177ef10>,
 <Metabolite cdpchol_r__Rhodosporidium at 0x32177e940>,
 <Metabolite cmp_r__Rhodosporidium at 0x32177e9d0>,
 <Metabolite h_r__Rhodosporidium at 0x32177e970>,
 <Metabolite pc_RT_r__Rhodosporidium at 0x321bce730>,
 <Metabolite h_e__Rhodosporidium at 0x321c40

and importantly the `medium`

In [118]:
community.medium

{'EX_co2_m': 1.99,
 'EX_h_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_nh4_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_pi_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_glc__D_m': 1.0,
 'EX_ca2_m': 999999.0,
 'EX_fe2_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_hco3_m': 1.99,
 'EX_k_m': 999999.0,
 'EX_leu__L_m': 1000.0,
 'EX_na1_m': 999999.0,
 'EX_no3_m': 1.76,
 'EX_mg2_m': 999999.0,
 'EX_mn2_m': 999999.0,
 'EX_cu2_m': 999999.0,
 'EX_zn2_m': 999999.0,
 'EX_cobalt2_m': 1.7e-05,
 'EX_mobd_m': 0.00016,
 'EX_photon410_m': 1000.0,
 'EX_photon430_m': 1000.0,
 'EX_photon450_m': 1000.0,
 'EX_photon470_m': 1000.0,
 'EX_photon490_m': 1000.0,
 'EX_photon510_m': 1000.0,
 'EX_photon530_m': 1000.0,
 'EX_photon550_m': 1000.0,
 'EX_photon570_m': 1000.0,
 'EX_photon590_m': 1000.0,
 'EX_photon610_m': 1000.0,
 'EX_photon630_m': 1000.0,
 'EX_photon650_m': 1000.0,
 'EX_photon670_m': 1000.0,
 'EX_photon690_m': 1000.0}

This behavior mimics the medium in `cobrapy`, but combines both models mediums into 1

### Running Optimization

Now that we have the model loaded, we can run standard `FBA` methods using `optimize()`. Default optimize does not return any fluxes from the model, so we can set the `fluxes=True` when calling the method to return them.

In [18]:
result = community.optimize(fluxes=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Rhodosporidium,0.5,2.432312,2398,2051
Synechococcus,0.5,2.000000,851,768
medium,NaN,NaN,285,285


We can see that both organisms have a non-zero growth rate and that the community growth is also non-zero. Let's check the fluxes.

In [171]:
result.fluxes.EX_sucr_e

compartment
Rhodosporidium   -0.256684
Synechococcus     0.256684
medium                 NaN
Name: EX_sucr_e, dtype: float64

#### Changing parts of the medium to test it's effect on growth

Now that we can successfully optimize the community model, we can begin altering the models medium and seeing how it changes the (community) growth rate.

First, let's make a copy of the original medium so that we can restore it after making changes.

In [25]:
medium_bkp = community.medium

Now we can make changes to the medium. The following cell is meant to be re-run with making changes. It will first restore the medium to the original and them set 

In [43]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_glc__D_m"] = 0
medium_to_change["EX_sucr_m"] = 0
medium_to_change["EX_leu__L_m"] = 1

# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

{'EX_co2_m': 1.99,
 'EX_h_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_nh4_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_pi_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_fe2_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_hco3_m': 1.99,
 'EX_k_m': 999999.0,
 'EX_leu__L_m': 1,
 'EX_na1_m': 999999.0,
 'EX_no3_m': 1.76,
 'EX_mg2_m': 999999.0,
 'EX_mn2_m': 999999.0,
 'EX_cu2_m': 999999.0,
 'EX_zn2_m': 999999.0,
 'EX_cobalt2_m': 1.7e-05,
 'EX_mobd_m': 0.00016,
 'EX_photon410_m': 1000.0,
 'EX_photon430_m': 1000.0,
 'EX_photon450_m': 1000.0,
 'EX_photon470_m': 1000.0,
 'EX_photon490_m': 1000.0,
 'EX_photon510_m': 1000.0,
 'EX_photon530_m': 1000.0,
 'EX_photon550_m': 1000.0,
 'EX_photon570_m': 1000.0,
 'EX_photon590_m': 1000.0,
 'EX_photon610_m': 1000.0,
 'EX_photon630_m': 1000.0,
 'EX_photon650_m': 1000.0,
 'EX_photon670_m': 1000.0,
 'EX_photon690_m': 1000.0}

Now that the medium is changed, we can rerun the model optimization.

In [44]:
result_altered_medium = community.optimize(fluxes=True)

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Rhodosporidium,0.5,0.000083,2398,2051
Synechococcus,0.5,0.458828,851,768
medium,NaN,NaN,285,285


In [55]:
result_altered_medium.fluxes.T.loc['EX_sucr_e']

compartment
Rhodosporidium   -7.23459
Synechococcus     7.23459
medium                NaN
Name: EX_sucr_e, dtype: float64

## Running the Models with MICOM Grow

An alternative to running standard community optimization with `optimize()`, we can also use a `micom.workflows` method called `grow()`. This simulates growth of the organism while also simulating potential tradeoffs (between prioritizing community vs. individual growth). This method does not require our previously constructed `community` object, but rather the `manifest` we added earlier.

A key difference here though, is that we need to create a `DataFrame` detailing the reaction, flux, and metabolite as the medium provided to the method.

### Building the Medium

In [111]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
grow_medium_to_change = community.medium

#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
grow_medium_to_change["EX_sucr_m"] = 1
#grow_medium_to_change["EX_leu__L_m"] = 1


In [112]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

,reaction,flux
0,EX_co2_m,1.990000
1,EX_h_m,999999.000000
2,EX_h2o_m,999999.000000
3,EX_nh4_m,999999.000000
4,EX_o2_m,999999.000000
5,EX_pi_m,999999.000000
6,EX_so4_m,999999.000000
7,EX_glc__D_m,1.000000
8,EX_ca2_m,999999.000000
9,EX_fe2_m,999999.000000


In [99]:
result_grow = grow(manifest, model_folder="models", medium=grow_medium, tradeoff=0.01, threads=2,presolve=True)

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmp3lurj44t.lp
Reading time = 0.02 seconds
: 3107 rows, 7069 columns, 28615 nonzeros


In [110]:
result_grow.exchanges

,taxon,sample_id,tolerance,reaction,flux,abundance,metabolite,direction
6,medium,One,0.000001,EX_zn2_m,-0.000786,NaN,zn2_m,import
21,medium,One,0.000001,EX_glyclt_m,0.006010,NaN,glyclt_m,export
23,medium,One,0.000001,EX_nh4_m,-1.799521,NaN,nh4_m,import
43,medium,One,0.000001,EX_photon670_m,-50.000000,NaN,photon670_m,import
50,medium,One,0.000001,EX_photon570_m,-50.000000,NaN,photon570_m,import
...,...,...,...,...,...,...,...,...
587,Synechococcus,One,0.000001,EX_so4_e,-0.000218,0.5,so4_e,import
588,Synechococcus,One,0.000001,EX_nh4_e,-0.010533,0.5,nh4_e,import
590,Synechococcus,One,0.000001,EX_photon590_e,-100.000000,0.5,photon590_e,import
593,Synechococcus,One,0.000001,EX_leu__L_e,-0.000736,0.5,leu__L_e,import
